## Coordinate frame transformations

This example shows how to perform transformations with different coordinate frames. 

In [29]:
import numpy as np
from vpython import *

In [30]:
# Create a canvas in this cell to display the output of vpython. All updates will be made here. 
canvas()

<IPython.core.display.Javascript object>

In [31]:
def Rz(angle):
    """
    3-D Rotation (about the z-axis)
    """
    return np.array( [[ cos(angle),  -sin(angle),   0 ],
                      [ sin(angle),   cos(angle),   0 ],
                      [          0,            0,   1 ]])

In [32]:
def Rt( R, t ):
    """
    Rigid-body transformation (homogeneous matrix)  [ R t ]
                                                    [ 0 1 ]
    where:

    R: 3x3 rotation matrix
    t: 3x1 translation vector
    """

    return np.block([[   R,    t],
                     [0, 0, 0, 1]])

In [33]:
def DrawFrame(f):
    """
    Draw coordinate frame using cylinders
    """
    # Cylinders representing the coordinate frame
    x_axis = cylinder(
        # Translation vector of the frame matrix
        pos    = vector(f[0,3],f[1,3],f[2,3]),
        # x-axis of the frame is the 1st column of the rotation matrix of the matrix frame.
        axis   = vector(f[0,0],f[1,0],f[2,0]),
        radius = 0.03,
        color  = vector(1,0,0)
        )

    y_axis = cylinder(
        # Translation vector of the frame matrix
        pos    = vector(f[0,3],f[1,3],f[2,3]),
        # y-axis of the frame is the 2nd column of the rotation matrix of the matrix frame.
        axis   = vector(f[0,1],f[1,1],f[2,1]),
        radius = 0.03,
        color  = vector(0,1,0)
        )

    z_axis = cylinder(
        # Translation vector of the frame matrix
        pos    = vector(f[0,3],f[1,3],f[2,3]),
        # y-axis of the frame is the 3rd column of the rotation matrix of the matrix frame.
        axis   = vector(f[0,2],f[1,2],f[2,2]),
        radius = 0.03,
        color  = vector(0,0,1)
        )

Each frame is described by a rigid-body transformation matrix. The matrix of Frame $j$ with respect to Frame $i$ is given by: 
$$
\begin{align}
           T_{i,j} =            
           \begin{bmatrix}
           	 R_{i,j} &{\bf t}_{i,j}\\
                 {\bf 0}_{1x3} & 1 
           \end{bmatrix},                   
	\label{T}
\end{align}
$$
where $R$ is a rotation matrix and ${\bf t}$ is a translation vector. The translation vector is the origin of the coordinate frame. 

In many applications, we need to use multiple local coordinate frames that are related to each other and also related to a global coordinate frame. For simplicity, each local frame can be described first by a transformation written with respect to a chosen frame of reference. If we have a sequence of local frames then it is common to represent each local frame with respect to its previous frame. Once we have all the local frames written with respect to their previous counterpart, we can then perform change of coordinates to convert the local representation to a global one as needed. 

### Frame $\{0\}$ (the global frame)

The global frame or Frame $\{0\}$ is an orthornormal frame centered at the origin of the global coordinate system. Its homogeneous matrix is the identity matrix, i.e.: 
$$
\begin{align}
           T_{00} =            
           \begin{bmatrix}
           	 I_{3\times3} &{\bf 0}_{3\times1}\\
                 {\bf 0}_{1x3} & 1 
           \end{bmatrix}.                   
	\label{T00}
\end{align}
$$



In [34]:
# Transformation defining frame {0} (i.e., global frame).
# Each column is an axis of the coordinate
# frame (i.e., column 1 is the x-axis, column 2 is the y-axis,
# and column 3 is the z-axis).
T00 = Rt( np.identity(3), np.zeros((3, 1)) )
print('\nFrame {0} = \n', T00)

# Draw the frame
DrawFrame(T00)


Frame {0} = 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


### Frame $\{1\}$

Let's now create another coordinate frame. The (local) transformation that represents Frame $\{1\}$ with respect to Frame $\{0\}$ is given by:  
$$
\begin{align}
           T_{0,1} =            
           \begin{bmatrix}
           	 R_{0,1} &{\bf t}_{0,1}\\
                 {\bf 0}_{1x3} & 1 
           \end{bmatrix}.                   
\end{align}
$$



Here, we will place this frame away from the previous frame, at a position ${\bf t}_{0,1} = \left(l, 0, l\right)$. We can place it anywhere we want. In the example shown here, Frame $\{1\}$ will not be rotated with respect to Frame $\{0\}$. 

In [35]:
# Create another coordinate frame (local transformation matrix for the frame w.r.t. previous frame)
l = 2                               # Translation distance along each axis 
R01 = Rz(0)                         # Rotation matrix
t01 = np.array( [[l], [0.0], [l]] ) # Translation vector (origin of the frame)
T01 = Rt(R01, t01)                  # Create (local) rigid-body transformation
print('\nT01 = \n', T01)


T01 = 
 [[ 1. -0.  0.  2.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0.  1.]]


### Converting from local to global coordinates
Prior to displaying each local frame, we must convert their matrix from a local to a global representation. This conversion is done by recursively multiplying the local frames in the order that they appear in the kinematic chain. The general conversion is given by: 
$$
\begin{align}
           T_{0,i} = \prod_{i=1}^{n} T_{i-1,i}.         
\end{align}
$$
i.e., 

$$
T_{0,1} = T_{0,1}, \\
T_{0,2} = T_{0,1}T_{1,2},\\
T_{0,3} = T_{0,1}T_{1,2}T_{2,3},\\
\dots\\
T_{0,n} = T_{0,1}T_{1,2}T_{2,3} \dots T_{n-1,n} .\\
$$

In [36]:
# To draw frame, we need first to calculate the transformation matrix representing
# the frame in global coordinates. In the case of T01, its local matrix is already written 
# in terms of the global frame, and we do not need to convert it from local to global. 

print('\nFrame {1} = \n', T01)

# Draw the new frame
DrawFrame(T01)


Frame {1} = 
 [[ 1. -0.  0.  2.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0.  1.]]


### Frame $\{2\}$

In [37]:
# Now, we create  another local frame. We call it frame {2}.
# The transformation of this local frame is given w.r.t. previous
# frame (i.e., frame 1). This frame will be translated away from
# the previous frame along the y-axis direction. This new frame will also
# be rotated w.r.t. previous frame.
l = 1                               # Length of translation
R12 = Rz(pi/8)                      # Rotation matrix
t12 = np.array( [[0], [l], [0]] )   # Translation vector
T12 = Rt(R12, t12)                  # Create (local) rigid-body transformation
print('\nT12 =\n', T12)


T12 =
 [[ 0.92387953 -0.38268343  0.          0.        ]
 [ 0.38268343  0.92387953  0.          1.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [38]:
# To draw frame {2}, we need first to calculate the transformation matrix representing
# the frame in global coordinates, i.e., we need to calculate T02.
T02 = np.dot(T01,T12)
print('\nFrame {2} = \n', T01)

# Draw the new frame
DrawFrame(T02)


Frame {2} = 
 [[ 1. -0.  0.  2.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  2.]
 [ 0.  0.  0.  1.]]


### Frame $\{3\}$

In [39]:
# Now, we create  another local frame.
# The transformation of this local frame is given w.r.t. previous
# frame. This frame will be translated away from
# the previous frame along the y-axis direction. This new frame will also
# be rotated w.r.t. previous frame.
l = 1                               # Length of translation
R23 = Rz(pi/8)                      # Rotation matrix
t23 = np.array( [[0], [l], [0]] )   # Translation vector
T23 = Rt(R23, t23)                  # Create (local) rigid-body transformation

In [40]:
# To draw frame {3}, we need first to calculate the transformation matrix representing
# the frame in global coordinates, i.e., we need to calculate T03.
T03 = np.dot(T02,T23)
print('\nFrame {3} = \n', T03)

# Draw the new frame
DrawFrame(T03)


Frame {3} = 
 [[ 0.70710678 -0.70710678  0.          1.61731657]
 [ 0.70710678  0.70710678  0.          1.92387953]
 [ 0.          0.          1.          2.        ]
 [ 0.          0.          0.          1.        ]]
